In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn import metrics

#from sklearn.decomposition import PCA
from scipy.stats import zscore
import matplotlib.pyplot as plt 

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [3]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
from sklearn.datasets import load_boston

data = load_boston()

# print(data.DESCR)

dfboston =  pd.DataFrame(data.data, columns=data.feature_names)
dfboston2 = pd.DataFrame(data.target, columns=['MEDV'])
Boston = pd.concat([dfboston,dfboston2],axis = 1)

In [6]:
Boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [15]:
print(data.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [16]:
Boston.shape

(506, 14)

In [17]:
Boston.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [18]:
X =  Boston.drop(["MEDV", "INDUS", "AGE"], axis = 1)
y =  Boston.pop("MEDV")

#### LINEAR REGRESSION

In [19]:
LR = LinearRegression()

#### BAGGED LINEAR REGRESSION

In [20]:
bag_LR = BaggingRegressor(base_estimator= LR, n_estimators=50,random_state=0)

#### BOOSTED LINEAR REGRESSION

In [21]:
boost_LR = AdaBoostRegressor(base_estimator= LR, n_estimators=50,random_state=0)

#### BAGGED DECISION TREE

In [22]:
bag_DT = BaggingRegressor(n_estimators=50,random_state=0)

#### BOOSTED DECISION TREE

In [23]:
boost_DT = AdaBoostRegressor(n_estimators=50,random_state=0)

#### KNN

In [24]:
KNN = KNeighborsRegressor(3)

#### BAGGED KNN

In [25]:
bag_KNN_model = BaggingRegressor(base_estimator=KNN, n_estimators = 80, random_state = 0)

### Bagged models

In [26]:
bag_LR = BaggingRegressor(base_estimator= LR, n_estimators=50,random_state=0)
bag_DT = BaggingRegressor(n_estimators=50,random_state=0)
bag_KNN = BaggingRegressor(base_estimator=KNN, n_estimators = 80, random_state = 0)

### Boosted models

In [27]:
boost_LR = AdaBoostRegressor(base_estimator= LR, n_estimators=50,random_state=0)
boost_DT = AdaBoostRegressor(n_estimators=50,random_state=0)


# Gradient_boost

boost_GB = GradientBoostingRegressor(n_estimators = 100)

In [28]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

from sklearn.metrics import roc_curve,auc
for model, name in zip([LR, bag_LR, boost_LR, KNN, bag_KNN, bag_DT, boost_DT, boost_GB ],
    ['LR', 'LR_bag', 'LR_boost', 'KNN', 'KNN_bag', 'DT_bag', 'DT_boost', 'GB_boost']):
    rmse = []
    for train, test in kf.split(X,y):
        xtrain, xtest = X.iloc[train,:], X.iloc[test,:]
        ytrain, ytest = y.iloc[train], y.iloc[test]
        model.fit(xtrain, ytrain)
        y_predict = model.predict(xtest)
        mse = np.sqrt(np.sum((ytest-y_predict) **2) / len(ytest)) # mse = metrics.mean_squared_error(ytest,y_predict)
        rmse.append(mse)                         # rmse.append(np.sqrt(mse))
    print("RMSE : %0.02f (+/- %0.5f) [%s]" % (np.mean(rmse), np.var(rmse, ddof = 1), name))


RMSE : 4.83 (+/- 0.53924) [LR]
RMSE : 4.83 (+/- 0.54884) [LR_bag]
RMSE : 5.17 (+/- 0.35819) [LR_boost]
RMSE : 5.84 (+/- 1.01962) [KNN]
RMSE : 5.58 (+/- 0.87368) [KNN_bag]
RMSE : 3.76 (+/- 1.08746) [DT_bag]
RMSE : 4.22 (+/- 0.75486) [DT_boost]
RMSE : 3.48 (+/- 0.68511) [GB_boost]
